In [2]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.naive_bayes
import sklearn.ensemble as ske
import sklearn.neighbors
import sklearn.pipeline
import sklearn.linear_model
import joblib
from sklearn import preprocessing
from sklearn import tree, linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel
import sys
import sqlite3
import importlib

In [3]:
df_wild = pd.read_pickle('/content/drive/MyDrive/học tập/DACN/wild.pickle')

In [4]:
print(df_wild.shape)

(50724, 56555)


In [5]:
df_wild.head()

,sample_sha1,benign,malicious,packed,unpacked_sample_sha1,unpacked_sample_id,packer_name,imp_setupremovefilelogentrya,imp_vardeccmp,imp_setdlgitemint,...,opcode_xor sub mov,opcode_xor sub mov add,opcode_xor test,opcode_xor test je,opcode_xor xor,opcode_xor xor mov,opcode_xor xor pop,opcode_xor xor pop mov,opcode_xor xor xor,opcode_xor xor xor pop
1,a2aac23e17be570c647fd22080e05b0e58449565,True,False,True,-1,-1,none,False,False,False,...,0.000022,0.0,0.000088,0.000084,0.000070,0.000066,0.000028,0.0,0.000053,0.0
2,c3aa534bbd20f3a3dd8e7a457f625291af9d0fe4,False,True,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000309,0.000000,0.000000,0.0,0.000331,0.0
3,bea3fb5a68ac91258e8339a8aa037304ae24242b,True,False,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
10,15cb9eab6b0f0ccd718a9235f07046dff100789f,False,True,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
12,8792cf08950abada1572b47948e86c9fce26c01a,True,False,True,-1,-1,none,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0


Chia nhỏ dataset

In [6]:
#Chọn ngẫu nhiên 12,000 mẫu
df = df_wild.sample(n=12000, random_state=102)

Sử lý dataset

In [7]:
# Loại bỏ các lables không cần thiết
df = df.drop(['sample_sha1','unpacked_sample_sha1','unpacked_sample_id'], axis=1)
df = df.drop(['benign_vt','malicious_vt'], axis=1)
df = df.drop(['benign','packer_name','source','similarity','most_similar_sha1','unpacked_similarity'], axis=1)


In [8]:
#Xử lý các cột có dạng object
for col in df.columns:
  dtype = df[col].dtype
  if dtype == object:
    le = preprocessing.LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [9]:
df.head()

,malicious,packed,imp_setupremovefilelogentrya,imp_vardeccmp,imp_setdlgitemint,imp_?append@?$basic_string@du?$char_traits@d@std@@v?$allocator@d@2@@std@@qaeaav12@abv12@ii@z,imp_zwwritefile,imp_glstencilop,imp_comparelinkinforeferents,imp__setmbcp,...,opcode_xor sub mov,opcode_xor sub mov add,opcode_xor test,opcode_xor test je,opcode_xor xor,opcode_xor xor mov,opcode_xor xor pop,opcode_xor xor pop mov,opcode_xor xor xor,opcode_xor xor xor pop
167256,True,True,False,False,False,False,False,False,False,False,...,0.000087,0.0,0.000329,0.000267,0.000086,0.00000,0.0,0.0,0.000000,0.0
14422,True,True,False,False,False,False,False,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0
185351,True,True,False,False,False,False,False,False,False,False,...,0.000029,0.0,0.000105,0.000099,0.000085,0.00011,0.0,0.0,0.000043,0.0
53241,False,True,False,False,False,False,False,False,False,False,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0
38618,False,True,False,False,False,False,False,False,False,False,...,0.000000,0.0,0.000199,0.000124,0.000000,0.00000,0.0,0.0,0.000000,0.0


In [10]:
print(df.shape)

(12000, 56544)


Chia dataset thành 10000 tập để train và 2000 mẫu để test

In [11]:
df_train = df.sample(n=10000, random_state=102)
df_test = df.drop(df_train.index)

In [12]:
#Vì tập dữ liệu không có Unpacked Malware, chỉ cần loại bỏ Packed Benign
data = df_train.drop(df_train[(df_train['malicious'] == False) & (df_train['packed'] == True)].index)

In [13]:
#Kiểm tra số lượng mẫu
Unpacked_Benign = data[(data['malicious'] == False) & (data['packed'] == False)].shape[0]
Packed_Malware = data[(data['malicious'] == True) & (data['packed'] == True)].shape[0]
print('Unpacked Benign: %i samples' % Unpacked_Benign)
print('Packed Malware: %i samples' % Packed_Malware)
print('Total TRAIN set: %i samples'% (Unpacked_Benign+Packed_Malware))


Unpacked Benign: 851 samples
Packed Malware: 6614 samples
Total TRAIN set: 7465 samples


In [15]:
#Chia tỉ lệ 50 - 50
data_PM = data[(data['malicious'] == True) & (data['packed'] == True)].sample(n=Packed_Malware-Unpacked_Benign, random_state=102)
data = data.drop(data_PM.index)

In [17]:
#Số lượng tập train
Unpacked_Benign = data[(data['malicious'] == False) & (data['packed'] == False)].shape[0]
Packed_Malware = data[(data['malicious'] == True) & (data['packed'] == True)].shape[0]
print('Unpacked Benign: %i samples' % Unpacked_Benign)
print('Packed Malware: %i samples' % Packed_Malware)
print('Total TRAIN set: %i samples'% (Unpacked_Benign+Packed_Malware))

Unpacked Benign: 851 samples
Packed Malware: 851 samples
Total TRAIN set: 1702 samples


In [18]:
#Xử lý tập train
y = data['malicious'].values
data = data.drop(['malicious','packed'], axis=1)

Xử lý tập test

In [19]:
Unpacked_Benign = df_test[(df_test['malicious'] == False) & (df_test['packed'] == False)].shape[0]
Packed_Malware = df_test[(df_test['malicious'] == True) & (df_test['packed'] == True)].shape[0]
Packed_Benign = df_test[(df_test['malicious'] == False) & (df_test['packed'] == True)].shape[0]

df_test_PM = df_test[(df_test['malicious'] == True) & (df_test['packed'] == True)].sample(n=(Packed_Malware-(Unpacked_Benign+Packed_Benign)), random_state=102)
df_test = df_test.drop(df_test_PM.index)

Unpacked_Benign = df_test[(df_test['malicious'] == False) & (df_test['packed'] == False)].shape[0]
Packed_Malware = df_test[(df_test['malicious'] == True) & (df_test['packed'] == True)].shape[0]
Packed_Benign = df_test[(df_test['malicious'] == False) & (df_test['packed'] == True)].shape[0]

print('Unpacked Benign: %i samples' % Unpacked_Benign)
print('Packed Malware: %i samples' % Packed_Malware)
print('Packed Benign: %i samples' % Packed_Benign)
print('Total TEST set: %i samples'% (Unpacked_Benign+Packed_Malware+Packed_Benign))

Unpacked Benign: 163 samples
Packed Malware: 680 samples
Packed Benign: 517 samples
Total TEST set: 1360 samples


In [20]:
#Chọn lọc features
print('Researching important feature based on %i total features' % data.shape[1])
fsel = ske.ExtraTreesClassifier().fit(data, y)

Researching important feature based on 56542 total features


In [21]:
#Tạo mô hình và chia tập train - test
model = SelectFromModel(fsel, prefit=True)

X_train = model.transform(data)
X_test = model.transform(df_test.drop(['malicious','packed'], axis=1))
y_train = y
y_test = df_test['malicious'].values

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [22]:
#In ra số lượng features chọn được
nb_features = X_train.shape[1]
features = []
print('%i features identified as important:' % nb_features)

indices = np.argsort(fsel.feature_importances_)[::-1][:nb_features]
print(indices)

6035 features identified as important:
[41310 41827 41806 ... 28657 54928 32717]


In [24]:
#In ra chỉ số quan trọng của từng features
for f in range(nb_features):
    print("%d. feature %s (%f)" % (f + 1, data.columns[indices[f]], fsel.feature_importances_[indices[f]]))

#Gán vào tập features để xử lý mô hình
for f in sorted(np.argsort(fsel.feature_importances_)[::-1][:nb_features]):
    features.append(data.columns[f])

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
1036. feature imp_strncmp (0.000155)
1037. feature string_b'.text' (0.000155)
1038. feature ngram_b'r\x00i\x00t\x00' (0.000155)
1039. feature ngram_b'  </tr' (0.000155)
1040. feature string_b't\t<Vt' (0.000155)
1041. feature string_b'\tw%9' (0.000155)
1042. feature imp__commode (0.000155)
1043. feature ngram_b'\x03\x00\x00\x00(\x00' (0.000154)
1044. feature opcode_dec jne mov (0.000154)
1045. feature ngram_b'4\x00b\x000\x00' (0.000154)
1046. feature ngram_b'l\x93g\xd7(\xdc' (0.000154)
1047. feature ngram_b'\x00\x00\xa9\x00 \x00' (0.000154)
1048. feature imp_queryperformancecounter (0.000154)
1049. feature opcode_mov mov movzx add (0.000154)
1050. feature string_b'Left' (0.000154)
1051. feature imp_endpaint (0.000153)
1052. feature string_b'_^]3' (0.000153)
1053. feature pesection_3_virtualAddress (0.000153)
1054. feature opcode_jne mov jmp (0.000153)
1055. feature opcode_lea mov mov movzx (0.000153)
1056. feature opcode_test je p

In [25]:
#Chọn 4 thuật toán
algorithms = {
        "DecisionTree": tree.DecisionTreeClassifier(max_depth=20),
        "RandomForest": ske.RandomForestClassifier(n_estimators=100),
        "GradientBoosting": ske.GradientBoostingClassifier(n_estimators=100),
        "AdaBoost": ske.AdaBoostClassifier(n_estimators=200)
    }
results = {}

#Train - test mô hình với lần lượt 4 thuật toán
print("Now testing algorithms")
for algo in algorithms:
    clf = algorithms[algo]
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print("%s : %f %%" % (algo, score*100))
    results[algo] = score

Now testing algorithms
DecisionTree : 83.897059 %
RandomForest : 87.647059 %
GradientBoosting : 86.911765 %
AdaBoost : 87.720588 %


In [26]:
#Chọn mô hình có Acc lớn nhất
winner = max(results, key=results.get)
print('Algorithm with highest accuracy on train/test is %s with a %f %% success' % (winner, results[winner]*100))

Algorithm with highest accuracy on train/test is AdaBoost with a 87.720588 % success


In [27]:
#In ra các chỉ số đánh giá khác của mô hình
clf = algorithms[winner]
res = clf.predict(X_test)
mt = confusion_matrix(y_test, res)

print("True positive : %i " % (mt[1][1]))
print("True negative : %i " % (mt[0][0]))

print("False positive : %i " % (mt[0][1]))
print("False negative : %i " % (mt[1][0]))

True positive : 656 
True negative : 537 
False positive : 143 
False negative : 24 


In [28]:
report = classification_report(y_test, res)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

       False       0.96      0.79      0.87       680
        True       0.82      0.96      0.89       680

    accuracy                           0.88      1360
   macro avg       0.89      0.88      0.88      1360
weighted avg       0.89      0.88      0.88      1360

